Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from itertools import combinations
from collections import namedtuple, defaultdict
from random import choice
from copy import deepcopy

from tqdm.auto import tqdm
import numpy as np

/Users/paul-raphael/Documents/ci2023/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
State = namedtuple('State', ['x', 'o'])

In [3]:
MAGIC = [2, 7, 6, 9, 5, 1, 4, 3, 8]

In [4]:
def print_board(pos):
    """Nicely prints the board"""
    for r in range(3):
        for c in range(3):
            i = r * 3 + c
            if MAGIC[i] in pos.x:
                print('X', end='')
            elif MAGIC[i] in pos.o:
                print('O', end='')
            else:
                print('.', end='')
        print()
    print()

In [5]:
def win(elements):
    """Checks is elements is winning"""
    return any(sum(c) == 15 for c in combinations(elements, 3))

def state_value(pos: State):
    """Evaluate state: +1 first player wins"""
    if win(pos.x):
        return 1
    elif win(pos.o):
        return -1
    else:
        return 0
    
    

In [6]:
def random_game():
    trajectory = list()
    state = State(set(), set())
    available = set(range(1, 9+1))
    while available:
        x = choice(list(available))
        state.x.add(x)
        trajectory.append(deepcopy(state))
        available.remove(x)
        if win(state.x) or not available:
            break

        o = choice(list(available))
        state.o.add(o)
        trajectory.append(deepcopy(state))
        available.remove(o)
        if win(state.o):
            break
    return trajectory

In [7]:
value_dictionary = defaultdict(float)
hit_state = defaultdict(int)
epsilon = 0.001

for steps in tqdm(range(500_000)):
    trajectory = random_game()
    final_reward = state_value(trajectory[-1])
    for state in trajectory:
        hashable_state = (frozenset(state.x), frozenset(state.o))
        hit_state[hashable_state] += 1
        value_dictionary[hashable_state] = value_dictionary[
            hashable_state
        ] + epsilon * (final_reward - value_dictionary[hashable_state])

100%|██████████| 500000/500000 [01:19<00:00, 6306.85it/s]


In [8]:
sorted(value_dictionary.items(), key=lambda e: e[1], reverse=True)[:10]

[((frozenset({3, 4, 5, 6, 9}), frozenset({1, 2, 7, 8})), 0.9172760676239365),
 ((frozenset({2, 4, 5, 7, 8}), frozenset({1, 3, 6, 9})), 0.9166112881491261),
 ((frozenset({1, 2, 3, 4, 8}), frozenset({5, 6, 7, 9})), 0.9156040888197092),
 ((frozenset({2, 3, 6, 7, 9}), frozenset({1, 4, 5, 8})), 0.9143279649942075),
 ((frozenset({2, 5, 7, 8, 9}), frozenset({1, 3, 4, 6})), 0.9138983167341244),
 ((frozenset({4, 5, 6, 7, 9}), frozenset({1, 2, 3, 8})), 0.9138121288629873),
 ((frozenset({4, 5, 6, 7, 8}), frozenset({1, 2, 3, 9})), 0.9126838040690546),
 ((frozenset({3, 5, 6, 7, 9}), frozenset({1, 2, 4, 8})), 0.9125964004695241),
 ((frozenset({1, 2, 5, 7, 8}), frozenset({3, 4, 6, 9})), 0.9123336643739864),
 ((frozenset({2, 6, 7, 8, 9}), frozenset({1, 3, 4, 5})), 0.9117175379847842)]

In [9]:
len(hit_state)

5477